In [1]:
import sys
import os
import gym
import torch
from torch import nn
import numpy as np
import json

sys.path.append("../Models")
sys.path.append("../Train_utils")

from Simple_Model import Neural_Net_Actor_Critic,Neural_Net_module
from Actor_critic import Episodic_learning

In [2]:
res_dir="../Results/Debug"
desc=json.load(open(os.path.join(res_dir,"config.json")))


#PARSE ACTIVATION FUNCTIONS
desc["Actor_model_name"]["args"]["activators"]=[getattr(nn,act["name"])(**(act["args"])) for act in desc["Actor_model_name"]["args"]["activators"]]
desc["Critic_model_name"]["args"]["activators"]=[getattr(nn,act["name"])(**(act["args"])) for act in desc["Critic_model_name"]["args"]["activators"]]

Train=Episodic_learning(
    model=Neural_Net_Actor_Critic(
          Actor_model=Neural_Net_module(**(desc["Actor_model_name"]["args"])),
          Critic_model=Neural_Net_module(**(desc["Critic_model_name"]["args"])),
          norm=lambda x:0.5*(x.pow(2)),
          **(desc["agent"]["args"])
          ),
    env=gym.make(desc["environment"]["name"]),
    sch_f=lambda x: (1) if x<0.5 else 0,
    res_dir="/content/Deep_Reinforcement_Learning/Results/Debug",
    **(desc["framework"]["args"])
)

In [ ]:
Train.Train(1000,1000,(0.5),static=True,modified_reward=False)

  0%|                                                                                                                                   | 0/200 [00:00<?, ?it/s]/home/lambda/experiments/RL/DRL_/RLENV/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):

  0%|                                                                                                                          | 1/1000 [00:00<05:02,  3.30it/s]

DONE
DONE
Actor_loss 1.31902088
Critic_loss 1.00000405
Rewards mean 1.00000000 Rewards std 0.00000000 Rewards sum 26.00000000



  0%|                                                                                                                          | 1/1000 [00:00<05:02,  3.30it/s]

DONE
DONE
Actor_loss 1.59039765
Critic_loss 1.00000008
Rewards mean 1.00000000 Rewards std 0.00000000 Rewards sum 13.00000000



  0%|▎                                                                                                                         | 3/1000 [00:00<02:15,  7.37it/s]

...

DONE
DONE



 14%|█████████████████▏                                                                                                      | 143/1000 [00:15<01:20, 10.64it/s]

Actor_loss 1.29893700
Critic_loss 1.00000658
Rewards mean 1.00000000 Rewards std 0.00000000 Rewards sum 13.00000000



 14%|█████████████████▍                                                                                                      | 145/1000 [00:16<01:12, 11.80it/s]

DONE
DONE
Actor_loss 1.23917100
Critic_loss 1.00000961
Rewards mean 1.00000000 Rewards std 0.00000000 Rewards sum 13.00000000


In [ ]:
import matplotlib.pyplot as plt
list(Train.episodes_rewards.keys())
rewards=np.array(list(map(lambda k:np.sum(np.array(Train.episodes_rewards[k])),list(Train.episodes_rewards.keys()))))
plt.figure(figsize=(25,5))
plt.plot(rewards[:])


Train.episodes_losses.keys()
Ac_L=list(
    map(
        lambda k: list(map(lambda s:Train.episodes_losses[k][s]["Actor_loss"],
                                 list(Train.episodes_losses[k].keys()))),
              list(
                  Train.episodes_losses.keys()
              )[:]
              )
    )
Ac_L=np.array(Ac_L,dtype=list)

Ac_L_t=np.concatenate(Ac_L)
Ac_L_m=np.array(list(map(lambda l: np.mean(np.array(l)),Ac_L.tolist())))
Ac_L_s=np.array(list(map(lambda l: np.std(np.array(l)),Ac_L.tolist())))

#plt.figure(figsize=(25,5))
#plt.plot(Ac_L_t)
#plt.show()
plt.figure(figsize=(25,5))
plt.plot(np.arange(Ac_L_m.shape[0]),Ac_L_m)
plt.fill_between(np.arange(Ac_L_m.shape[0]),Ac_L_m-Ac_L_s, Ac_L_m+Ac_L_s, facecolor='blue', alpha=0.5)
plt.show()

Train.episodes_losses.keys()
Cr_L=list(
    map(
        lambda k: list(map(lambda s:Train.episodes_losses[k][s]["Critic_loss"],
                                 list(Train.episodes_losses[k].keys()))),
              list(
                  Train.episodes_losses.keys()
              )[:]
              )
    )
Cr_L=np.array(Cr_L,dtype=list)

Cr_L_t=np.concatenate(Cr_L)
Cr_L_m=np.array(list(map(lambda l: np.mean(np.array(l)),Cr_L.tolist())))
Cr_L_s=np.array(list(map(lambda l: np.std(np.array(l)),Cr_L.tolist())))

#plt.figure(figsize=(25,5))
#plt.plot(Cr_L_t)
#plt.show()
plt.figure(figsize=(25,5))
plt.plot(np.arange(Cr_L_m.shape[0]),Cr_L_m)
plt.fill_between(np.arange(Cr_L_m.shape[0]),Cr_L_m-Cr_L_s, Cr_L_m+Cr_L_s, facecolor='blue', alpha=0.5)
plt.show()


logit=Ac_L_t/Cr_L_t


logit_m=Ac_L_m/Cr_L_m


#plt.figure(figsize=(25,5))
#plt.plot(logit_t)
#plt.show()
plt.figure(figsize=(25,5))
plt.plot(np.arange(logit_m.shape[0]),logit_m)
plt.show()

In [ ]:
Actor=Neural_Net_module(
    state_size=4,
    action_size=2,
    #layer_sizes=[8,16,16,32,32,16,8,4],
    #activators=[nn.GELU(),nn.GELU(),nn.GELU(),nn.GELU(),nn.GELU(),nn.GELU(),nn.GELU(),nn.GELU(),nn.Softmax(dim=-1)],
    layer_sizes=[32,128,2],
    activators=[nn.GELU(),nn.GELU(),nn.GELU(),nn.Softmax(dim=-1)],
    )
Critic=Neural_Net_module(
    state_size=4,
    action_size=1,
    #layer_sizes=[8,16,16,32,32,16,8,4],
    layer_sizes=[32,128,32],
    #activators=[nn.GELU(),nn.GELU(),nn.GELU(),nn.GELU(),nn.GELU(),nn.GELU(),nn.GELU(),nn.GELU(),nn.Identity()],
    activators=[nn.GELU(),nn.GELU(),nn.GELU(),nn.Identity()],
    )


Model = Neural_Net_Actor_Critic(
    Actor_model=Actor,
    Critic_model=Critic,
    gamma=0.5,
    norm=lambda x:0.5*(x.pow(2))
    #norm=lambda x:torch.abs(x)
    )

env=gym.make("CartPole-v1")


Train=Episodic_learning(
    model=Model,
    free_input=0,
    env=env,
    Actor_optimizer_params={
        "lr":1e-5,
        "betas":(0.0, 0.9),
        "eps":1e-08,
        "weight_decay":0.99
        },
    Critic_optimizer_params={"lr":1e-7},
    phi=1e-1,  #self.phi*Cri_loss+(1-self.phi)*Act_loss
    res_dir="../Results/Debug",
    max_steps=200,
    multi_opt=False,
    cuda=False,
    #sch_f=lambda x:(0.5-2e-8)*np.cos(2*np.pi*x)+(0.5)
    #sch_f=lambda x: (1-1e-10) if x<0.5 else 1e-10
    sch_f=lambda x: (1) if x<0.5 else 0
)

In [ ]:
#Train.Train(200,500,1e-9,static=False)
Train.Train(500,200,(1e-5),static=True,modified_reward=True)

In [24]:
import matplotlib.pyplot as plt
list(Train.episodes_rewards.keys())
rewards=np.array(list(map(lambda k:np.sum(np.array(Train.episodes_rewards[k])),list(Train.episodes_rewards.keys()))))
plt.figure(figsize=(25,5))
plt.plot(rewards[:])


Train.episodes_losses.keys()
Ac_L=list(
    map(
        lambda k: list(map(lambda s:Train.episodes_losses[k][s]["Actor_loss"],
                                 list(Train.episodes_losses[k].keys()))),
              list(
                  Train.episodes_losses.keys()
              )[:]
              )
    )
Ac_L=np.array(Ac_L,dtype=list)

Ac_L_t=np.concatenate(Ac_L)
Ac_L_m=np.array(list(map(lambda l: np.mean(np.array(l)),Ac_L.tolist())))
Ac_L_s=np.array(list(map(lambda l: np.std(np.array(l)),Ac_L.tolist())))
Ac_L_min=np.array(list(map(lambda l: np.min(np.array(l)),Ac_L.tolist())))
Ac_L_max=np.array(list(map(lambda l: np.max(np.array(l)),Ac_L.tolist())))

#plt.figure(figsize=(25,5))
#plt.plot(Ac_L_t)
#plt.show()
plt.figure(figsize=(25,5))
plt.plot(np.arange(Ac_L_m.shape[0]),Ac_L_m)
plt.fill_between(np.arange(Ac_L_m.shape[0]),Ac_L_m-Ac_L_s, Ac_L_m+Ac_L_s, facecolor='blue', alpha=0.3)
plt.fill_between(np.arange(Ac_L_m.shape[0]),Ac_L_min, Ac_L_max, facecolor='blue', alpha=0.5)
plt.show()

Train.episodes_losses.keys()
Cr_L=list(
    map(
        lambda k: list(map(lambda s:Train.episodes_losses[k][s]["Critic_loss"],
                                 list(Train.episodes_losses[k].keys()))),
              list(
                  Train.episodes_losses.keys()
              )[:]
              )
    )
Cr_L=np.array(Cr_L,dtype=list)

Cr_L_t=np.concatenate(Cr_L)
Cr_L_m=np.array(list(map(lambda l: np.mean(np.array(l)),Cr_L.tolist())))
Cr_L_s=np.array(list(map(lambda l: np.std(np.array(l)),Cr_L.tolist())))
Cr_L_min=np.array(list(map(lambda l: np.min(np.array(l)),Cr_L.tolist())))
Cr_L_max=np.array(list(map(lambda l: np.max(np.array(l)),Cr_L.tolist())))

#plt.figure(figsize=(25,5))
#plt.plot(Cr_L_t)
#plt.show()
plt.figure(figsize=(25,5))
plt.plot(np.arange(Cr_L_m.shape[0]),Cr_L_m)
plt.fill_between(np.arange(Cr_L_m.shape[0]),Cr_L_m-Cr_L_s, Cr_L_m+Cr_L_s, facecolor='blue', alpha=0.1)
plt.fill_between(np.arange(Cr_L_m.shape[0]),Cr_L_min, Cr_L_max, facecolor='blue', alpha=0.5)
plt.show()


logit=Ac_L_t/Cr_L_t


logit_m=Ac_L_m/Cr_L_m


#plt.figure(figsize=(25,5))
#plt.plot(logit_t)
#plt.show()
plt.figure(figsize=(25,5))
plt.plot(np.arange(logit_m.shape[0]),logit_m)
plt.show()

In [25]:
2+2